In [2]:
import numpy as np
import pandas as pd
import datetime as dt

from bokeh.plotting import figure 
from bokeh.io import output_notebook, show
from bokeh.models import LabelSet, ColumnDataSource

output_notebook()

Loading BokehJS ...

In [3]:
df = pd.read_csv('rwc.csv', index_col=0, parse_dates=['Date'])
df

,Date,Type,Distance_km,Hours,Minutes,Seconds,Time_h,Calories,ElevGain_m,AvgSpeed_km/h,Year,Month
0,2015-07-14,Walking,2.10,0,26,40,0.444444,89.0,28.0,4.725000,2015,7
1,2015-07-20,Cycling,21.18,1,8,13,1.136944,332.0,270.0,18.628879,2015,7
2,2015-07-25,Cycling,23.52,1,9,32,1.158889,390.0,327.0,20.295302,2015,7
3,2015-07-27,Running,6.94,0,42,4,0.701111,389.0,97.0,9.898574,2015,7
4,2015-07-29,Walking,1.73,0,20,36,0.343333,69.0,32.0,5.038835,2015,7
...,...,...,...,...,...,...,...,...,...,...,...,...
292,2021-01-05,Running,10.18,0,55,43,0.928611,691.0,213.0,10.962608,2021,1
293,2021-01-08,Cycling,23.50,0,45,0,0.750000,520.0,NaN,31.333333,2021,1
294,2021-01-10,Running,10.18,0,55,9,0.919167,675.0,210.0,11.075249,2021,1
295,2021-01-12,Running,10.15,0,56,3,0.934167,689.0,211.0,10.865299,2021,1


In [51]:
runs = df.loc[df.Type=='Running'].groupby('Year').sum()

runs.Distance_km = runs.Distance_km.round(2)

color_dst, time_spent = [], []
for year in runs.index:
    
    # Colors
    if runs.Distance_km[year] == min(runs.Distance_km):
        color_dst.append('green')
    elif runs.Distance_km[year] == max(runs.Distance_km):
        color_dst.append('red')
    else:
        color_dst.append('blue')

    hour = int(runs.Time_h[year])
    minutes = int(round((runs.Time_h[year]-hour)*60, 0))
    time = str(hour)+'h '+str(minutes)+'min'
    time_spent.append(time)

runs['color_dst'] = color_dst
runs['time_spent'] = time_spent
runs['avg_speed'] = df.loc[df.Type=='Running'].groupby('Year').mean()['AvgSpeed_km/h'].round(2)

In [46]:
runs

,Distance_km,Hours,Minutes,Seconds,Time_h,Calories,ElevGain_m,AvgSpeed_km/h,Month,color_dst,time_spent,avg_speed_km/h
Year,,,,,,,,,,,,
2015,16.14,1,43,45,1.729167,924.0,97.0,18.847506,16,green,1h 44min,9.42
2016,30.09,0,191,82,3.206111,1785.0,608.0,37.839855,31,blue,3h 12min,9.46
2017,51.88,0,303,245,5.118056,2858.0,596.0,100.644617,99,blue,5h 7min,10.06
2018,142.96,4,624,496,14.537778,8526.0,2035.0,188.780794,124,blue,14h 32min,9.94
2019,441.57,4,2214,1560,41.333440,24411.0,6712.0,515.588548,265,red,41h 20min,10.74
2020,438.00,16,1487,1011,41.064167,25239.0,5264.0,397.226706,222,blue,41h 4min,10.74
2021,50.19,0,279,85,4.673611,3327.0,1112.0,53.705600,5,blue,4h 40min,10.74


In [53]:
source = ColumnDataSource(runs)

# Tooltips
tooltips = [('Distance', "@Distance_km{0,0.00} km"), ('Time', "@time_spent"),
            ("Calories burned","@Calories{0,0}"), ("Cumulative Elevation Gain", "@ElevGain_m{0,0} m"),
            ("Average Speed", "@avg_speed{0.00} km/h")]

# Instantiate the figure
sports_fig = figure(title='Distance Ran by Year', x_axis_label='Year', y_axis_label = 'Distance (km)',
                    tooltips=tooltips, plot_width=500, plot_height=500, tools='save')

# Tweak the title
sports_fig.title.align = 'center'
sports_fig.title.text_font_size = "20px"

# Remove gridlines
sports_fig.xgrid.grid_line_color, sports_fig.ygrid.grid_line_color = None, None

# Remove x axis minor ticks
sports_fig.xaxis.minor_tick_line_color = None

sports_fig.outline_line_color = None

# Vertical bars
sports_fig.vbar(x='Year', top='Distance_km', width=0.9, source=source, color='color_dst')

# Get the labels
labels = LabelSet(x='Year', y='Distance_km', text='Distance_km', level='glyph', text_align='center',
                  source=source, render_mode='canvas', y_offset=3)

# Add the labels to the figure
sports_fig.add_layout(labels)

# Show the figure
show(sports_fig)

In [33]:
LabelSet?